In [21]:
import math
import numpy as np
import nupack
from nupack import State, Model
import scipy.stats
from scipy.sparse.linalg import spsolve
from nupack import *
from nupack.rotation import unique_partitions

In [3]:
# KD//Same as in Small box vs. analytic

def structure_matrix(seq):
    lengd = len(seq)
    leftB = []
    rightB = []
    Smatrix = np.zeros([lengd,lengd])
    
    for i in range(lengd):
        if seq[i] == '(':
            leftB.append(i)
        elif seq[i] == ')':
            rightB.append(i)
        
    for x,y in list(enumerate(leftB)):
        Smatrix[y,rightB[len(rightB)-1-x]] = 1
        Smatrix[rightB[len(rightB)-1-x], y] = 1
    Smatrix += np.diag(1-Smatrix.sum(0))
    return Smatrix

In [5]:
# Byr til oll secondary structure og reiknar svo pi (stationary distribution)
# KD//translation: (something) ... secondary structure and then calculate stationary distribution 
# KD//(finds all secondary structures?)
seq = 'AAATTT'
#all_states = sorted(unbound_State.enumerated())
#small_pi = np.asarray([w.boltz for w in all_states])
#small_pi /= small_pi.sum()

mod = Model(T=298, gt = False)
#mod.beta = 0

state = lambda seq, dp=None: State(seq, dp, model=mod, kind='full')

unbound_State = state(seq)
all_states, small_pi, R = state(seq).rate_matrix()

print(R)    # KD//rate matrix?

AttributeError: 'State' object has no attribute '_rates'

In [23]:
w = state(seq)
print(w.pairs.dp())
w.step()
print(w.pairs.dp())

........


ConversionError: C++: wrong number of arguments (expected 1, got 2)

In [ ]:
# KD//WHAT IS "self"? current microstate? isn't this "w"? is "self" a class/object within a class 
# KD//to capture data relevent to finding Z?

# Z matrix (fundamental matrix), sja Fornace report og toflu 27.06.2017
# From page 34, doesn't work
class Fundamental_Observable:
    def __init__(self, all_states, small_pi, observable):
        self.observable = observable
        self.integral = 0.0
        self.all_states = all_states
        self.small_pi = small_pi
        self.pi_part = 0
        
    def __call__(self, state, timestep):
        self.pi_part = self.pi_part + self.small_pi * timestep
        self.integral = self.integral + self.observable(state) * timestep
    
    def result(self):
        return self.integral
    
class Delta_Observable:     # KD//What is this for?
    def __init__(self, all_states):
        self.all_states = all_states
        
    def __call__(self, state):
        collect = np.zeros(len(self.all_states))
        for x,y in enumerate(self.all_states):
            if y.dp == state.dp:
                collect[x] = 1
        return collect


class Stop_Condition:
    def __init__(self, stop_time):     # KD//how do we choose a stop time?
        self.time = 0.0
        self.stop_time = stop_time

    def __call__(self, state, timestep):
        self.time += timestep
        return self.time >= self.stop_time
    
Z = [0] * len(all_states)
repeats = 500                # KD//Why 500? Is this arbitrary? 
for i, w in enumerate(all_states):
    for n in range(repeats):
        obs = Fundamental_Observable(all_states, small_pi, Delta_Observable(all_states))
        state(w.sequence, w.dp).run(0.01, observers = [obs])     # KD//look up this function
        Z[i] += (obs.integral - obs.pi_part) / repeats     # KD//add incremental portions of the integral for each state
        #print(obs.integral, obs.pi_part, '\n')
Z = np.asarray(Z)
Z -= np.diag(Z.sum(1)) # KD//sum along the rows of Z, create a diagonal matrix of the sums, subtract off from Z--WHY??

#print(small_pi @ Z)
print('Z\n', Z)

In [ ]:
big_pi = np.asarray([small_pi] * len(small_pi))
scale = -R.diagonal().mean()     # KD//Why rescale R by this?
Z_exact = -(np.linalg.inv(R/scale + big_pi) - big_pi) / scale     # KD//scaling?

print('inverse R eigs\n', 1/np.linalg.eigvals(R))
#print('Z eigs\n', np.linalg.eigvals(Z))
print('Z exact eigs\n', np.linalg.eigvals(Z_exact))     # KD//why do we want the eigenvalues of Z?

print('Z exact\n', Z_exact)

In [ ]:
#def f(secondary_struc,all_states):
#    collect = np.zeros(len(all_states))
#    for x,y in enumerate(all_states):
#        if y.dp == secondary_struc.dp:
#            collect[x] = 1
#    return collect

In [ ]:
#def g(secondary_struc,all_states):
#    collect = np.zeros(len(all_states))
#    for x,y in enumerate(all_states):
#        if y.dp == secondary_struc.dp:
#            collect[x] = 1
#    return collect

In [ ]:
# Find S_f and S_g as on page 35, see table 27.07.17
S_f = 0
for i in range(len(R))
    S_f += -1/(R[i][i]) * delta(i) * f(X_s)
    


In [ ]:
S_g = 0
for i in range(len(R))
    S_f += -1/(R[i][i]) * delta(i) * g(X_s)
    

In [ ]:
#(E@S_f@S_g)/t = f(i)g(j)

In [ ]:
from nupack4 import Structure_Counter
seq = 'GCTTGAGATGTTAGGGAGTAGTGCTCCAATCACAACGCACTACTCCCTAACATC+GCACTACTCCCTAACATCTCAAGC+AGGGAGTAGTGCGTTGTGATTGGAAACATCTCAAGCTCCAATCACAACGCACTA+GTTGTGATTGGAGCTTGAGATGTTGCACTACTCCCTAACATCTCAAGCTCCAAT'
dp = '......((((((((((((((((((............))))))))))))))))))+........................+......((((((((((((((((((............))))))))))))))))))+......((((((((((((((((((............))))))))))))))))))'

w = nupack4.State('AAAAT', model=Model(T=296, gt = True), kind='slow')
all_states, small_pi, R = w.rate_matrix()
timer, counter = w.run(10.1, observers=[Structure_Counter()])
print(len(all_states))
print(timer)
print(counter)

In [ ]:
for w in all_states: counter.emplace(w.pairs)
print(keys[4])
print(all_states[1].pairs)

In [ ]:
# Find Z according to page 34 with C++ bindings
 

# Define sequence, temperature, gt pairs, etc. (define State)
# Find all possible states, stationary distribution and enumerated rate matrix
w = state(seq)
all_states, small_pi, R = w.rate_matrix()

w_tmp = state(all_states[1].sequence, all_states[1].dp)

print(-1/R[0,0])
x = w_tmp.run(time = 0.00001, observers=[time_integrator()])
#print(np.asarray(y.values()) / x[1].time)
#print(small_pi)
#print('steps per time', x[1].step / x[1].time)


def z_sample(w):
    w_tmp = state(w.sequence, w.dp)
    w_final, time, counter = w_tmp.run(time=0.001, observers=[time_integrator()])
    for w in all_states: counter.emplace(w.pairs)
    #structures = list(counter.keys())
    times = np.asarray(counter.values())
    return times - small_pi* time.time


Z_new = np.zeros([len(all_states)]*2)
repeats = 100
for i, w in enumerate(all_states):
    for n in range(repeats):
        Z_new[i] += z_sample(w)
    Z_new[i] /= repeats

Z_new -= np.diag(Z_new.sum(1))

print(Z_new)

In [ ]:
x[1]

In [ ]:
# Use the rate matrix found in 8) to solve the ODE and draw over a certain timescale.

import matplotlib.pyplot as plt
import seaborn as sns

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
          '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
          '#bcbd22', '#17becf']

sns.set(style='whitegrid', palette=colors, rc={'axes.labelsize': 16})

# The following is specific Jupyter notebooks
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}


def one_step(vec,t):
    dvecDt = vec@R
    return dvecDt

y0 = np.array([0,1])
t = np.linspace(0,1e-5,num = 10000)
ts = 1e-5/10000

vec = scipy.integrate.odeint(one_step ,y0 ,t)


# Smellir upp grunn
fig, ax = plt.subplots(1, 1)
# Setur merkingar a asa
ax.set_xlabel(r'$Time \ ( s )$')
ax.set_ylabel(r'$Dilution \ (nM)$');

_ = ax.plot(t, vec[:,0]/vec.sum(1))
_ = ax.plot(t, vec[:,1]/vec.sum(1))

ax.legend(('Macrostate 1', 'Macrostate 2'), loc='upper right')


In [ ]:
first = (vec[:,0].sum() * ts) - vec.sum() * ts * small_pi[0]

In [ ]:
second = (vec[:,1].sum() * ts) - vec.sum() * ts * small_pi[1]

In [ ]:
print(vec)

In [ ]:
Z = np.asarray([[-first,first],[-second,second]])

In [ ]:
print(Z)

In [ ]:
print(Z_exact)

In [ ]:
np.asarray([1,0]) - small_pi[0]

In [ ]:
n = len(small_pi)
S = np.zeros([n] * 2)
print(S)
t = np.linspace(0,1e-4,num = 100000)
#ts = 1e-5/10000

def one_step(vec,t):
    dvecDt = vec@R
    return dvecDt

for start in range(n):
    x0 = np.zeros([n])
    x0[start] = 1
    vec = scipy.integrate.odeint(one_step, x0 ,t)
    for i in range(n):
        S[start, i] = np.trapz(vec[:, i] - small_pi[i], t)

print(S)
        
Gamma = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        Gamma[i, j] = sum(small_pi[start] * S[start, i] * S[start, j] / t[-1] for start in range(n))

print(Gamma)

In [ ]:
# Page 35 in book, see board (01.08.17)
# Find fundamental matrix, Z
Y = np.zeros([n] * 3)
reps = 1000             # How often the inner for-loop should run

def one_step(vec,t):
    dvecDt = vec@R
    return dvecDt

for start in range(n): # for-loop over all the indexes for the states
    start_state = all_states[start]   #starting states defined as the one with index "start"
    for r in range(reps):
        _, time, counter = start_state.run(time=1e-5, observers=[time_integrator()])
        for w in all_states: counter.emplace(w.pairs)
        times = np.asarray(counter.values())
        times -= time.time * small_pi
        Y[start] += np.outer(times, times) / reps / time.time           # (*) devided by t

Gamma = sum(small_pi[start] * Y[start] for start in range(n))
Z_35 = np.diag(1/small_pi) @ Gamma/2
#Z_35 = np.asarray([Gamma[i, :] / 2 / small_pi[i] for i in range(n)])
print(Gamma)
print(Z_35)

In [ ]:
# Same way as above, except that: (01.08.2017)
# f and g not as delta functions
f = np.arange(n)+1.
g = np.cos(np.arange(n))
f -= f @ small_pi
g -= g @ small_pi

fg= np.array([f, g])
Y = np.zeros([n] * 3)
reps = 10000

for start in range(n):
    start_state = all_states[start]
    for r in range(reps):
        _, time, counter = start_state.run(time=1e-4, observers=[time_integrator()])
        for w in all_states: counter.emplace(w.pairs)
        times = np.asarray(counter.values())
        #times -= time.time * small_pi
        Y[start] += np.outer(fg @ times, fg @ times) / reps / time.time

Gamma_new = sum(small_pi[start] * Y[start] for start in range(n))
Z_35_new = np.diag(1/small_pi) @ Gamma_new/2

exact = 2 * fg @ np.diag(small_pi) @ Z_exact @ fg.T

print(exact)
print(Gamma_new)

In [ ]:
np.diag(small_pi) @ Z_exact

In [ ]:
# F and G not delta functions and with killing/weighting terms
f = np.arange(n)+1.
g = np.cos(np.arange(n))
f -= f @ small_pi
g -= g @ small_pi

fg= np.array([f, g])
warmup = 1e-5
all_pairs = tuple(w.pairs for w in all_states)

def do_one():
    start_state = all_states[0]
    _, time, counter = start_state.run(time=1e-4, observers=[covariance_integrator(all_pairs, warmup)])
    times = np.asarray(counter.times)
    return counter.products / (counter.time - warmup)

reps = 10000
matrix = sum(do_one() for _ in range(reps)) / reps

print('Matrix\n', matrix)

Gamma_new = fg @ matrix @ fg.T 
print('Sum', Gamma_new.sum())

#Z_35_new = np.diag(1/small_pi) @ Gamma_new/2

exact = 2 * fg @ np.diag(small_pi) @ Z_exact @ fg.T

print('Gamma\n', Gamma_new)

print('Exact\n', exact)

print('Ratio')
print(Gamma_new / exact)

In [ ]:
print('total times', counter.times)
print('~ covariance\n', counter.products / (counter.time - warmup))
print('time', counter.time)

In [ ]:
# Nytt verkefni sett fyrir a toflu 09.08.17

evals, evecs = scipy.linalg.eigh(np.diag(small_pi)@R,np.diag(small_pi))
print(evecs.T@np.diag(small_pi)@evecs)
print(np.diag(np.ones(len(small_pi))))



In [ ]:
G = evecs.T@np.diag(small_pi)@R@evecs
G0 = np.where(np.abs(G) > 1.e-5, G, 0)
print(G0)

f = evecs[:, 1:3].T
print(f.shape)

In [ ]:
pairs = [w.pairs for w in all_states]
obs = scaled_covariance_integrator(pairs, f, 1e5, 0.01)

w, time, cov = all_states[0].run(time=10.5, observers=[obs])

In [ ]:
print(cov.products/10.5)

In [ ]:
###################################################################################################
###################################################################################################

# 


In [ ]:
import numpy as np
from scipy.linalg import eigh
from nupack4 import *
 
mod = Model(dangles='none')
w = State('GUCGCGUCGCGUCGCUAUGCGAC',kind='slow',model=mod)
#w = State('AAATTT',kind='slow',model=mod)

def unique_states(states):
    tmp = []
    for j in range(len(states)):
        if all(states[j]!= tmp[k] for k in range(len(tmp)) ):
            tmp.append(states[j])
    return tmp
        
def sample(n, eng, seq, model):
    return [State(seq, s.dp([len(seq)]), model=model, kind='slow') for s in eng.sample(seq, n)]

    
basis_states = sample(w, 50)
#basis_states = w.rate_matrix()[0][:2]
basis_states = unique_states(basis_states)
    
launch_states = sample(w, 100)
    
basis_strucs = [y.pairs for y in basis_states]

n = len(basis_strucs)

print('Without duplicates:', n)
print(basis_states)

scale = 3e6
t = 5.e-4
A = slice(None)
B = slice(n, None)

Gamma = np.zeros([2*n, 2 *n])
time = 0.
Overlap = np.zeros([2*n, 2 *n])
Grad = np.zeros([2*n, 2 *n])
times = np.zeros([2*n])

for z in launch_states:
    #enum = enumerated_integrator(enumerated_observer(pairs, np.eye(n)), scale, 0)
    ham = hamming_integrator(hamming_observer(basis_strucs, 0), scale, 0)
    #cov = covariance_integrator(lambda w, t: np.ones([4]), scale, 0)
    z.run(time=t, observers=[ham])
    Gamma += ham.gamma
    time += ham.time
    times += ham.times
    print(ham.times)
    Overlap += ham.overlap
    Grad += ham.gradient
    
print(t, '\t', 1/scale)

#print(enum.gamma)
Overlap = Overlap[A, A] / time # Matrix
Overlap = 0.5 * (Overlap + Overlap.T)
print('Overlap\n', Overlap)

norms = np.sqrt(np.diagonal(Overlap))
print('Norms', norms)
Ham_Gamma = 1e6 * Gamma[A, A] / time  / np.outer(norms, norms) # Matrix
Ham_Gamma = 0.5 * (Ham_Gamma + Ham_Gamma.T)

Ham_Grad = 1e12 * Grad[A, A] / time / np.outer(norms, norms)
Ham_Grad = 0.5 * (Ham_Grad + Ham_Grad.T)
print('Gradient\n', Ham_Grad)

Norm_Overlap = Overlap / np.outer(norms, norms)

print('Normalized overlap\n', Norm_Overlap)

means = times[A] / time / norms # vector
iGamma = np.linalg.inv(Ham_Gamma)
new_pi = means * (iGamma @ means) # vector
eta = new_pi.sum()   # scalar
new_pi /= eta

#print('stuff', means @ (iGamma @ means), means @ iGamma @ means, (means * (iGamma @ means)).sum())
print('stationary state', new_pi.shape, new_pi.sum(), '\n', new_pi)
M = np.outer(means, means) / eta
Projected_Gamma = (np.eye(len(Ham_Gamma)) - iGamma @ M).T @ Ham_Gamma @ (np.eye(len(Ham_Gamma)) - iGamma @ M)

Ham_S = np.diag(1/means[B]) @ Projected_Gamma[B, B] @ np.diag(1/means[B])

gamma_pi = iGamma @ means / eta

Gamma_Pi = np.outer(gamma_pi, gamma_pi) / (gamma_pi @ gamma_pi)

T = np.linalg.inv(Projected_Gamma + Gamma_Pi) - Gamma_Pi

print('1st order T\n', T)
print('T correction\n', (scale / 1e6) * T @ Ham_Grad @ T)

Gamma2 = np.linalg.inv(Gamma_Pi + T + (scale/1e6) * T @ Ham_Grad @ T) - Gamma_Pi
print('Gamma eigs\n', np.linalg.eigvals(Projected_Gamma))
print('T eigs\n', np.linalg.eigvals(T))
print('Gamma2 eigs\n', np.linalg.eigvals(Gamma2))

print('------------------------------------------')
print('should be 0\n', Projected_Gamma @ gamma_pi)
print('-')
print(T@gamma_pi)
print('-')
print(Gamma2@gamma_pi)
print('------------------------------------------')

print('Approximate Gamma\n', Projected_Gamma)
print('Fixed Gamma\n', Gamma2)

Ham_S2 = np.diag(1/means[B]) @ Gamma2[B, B] @ np.diag(1/means[B])

print('Approximate S\n', Ham_S)
print('Fixed S\n', Ham_S2)
print('Approximate hitting time\n', -Ham_S + np.asarray([np.diagonal(Ham_S)]*len(Ham_S)))
print('Fixed hitting time\n', -Ham_S2 + np.asarray([np.diagonal(Ham_S2)]*len(Ham_S2)))

iGamma2 = np.linalg.inv(Gamma2[B, B])
new_pi2 = means[B] * (iGamma2 @ means[B]) # vector
new_pi2 /= new_pi2.sum()


approx = -Ham_S + np.asarray([np.diagonal(Ham_S)]*len(Ham_S))
not_approx = -Ham_S2 + np.asarray([np.diagonal(Ham_S2)]*len(Ham_S2))

print('New pi', new_pi2)
print('------------------------------')
#prentugaur1 = np.zeros[2,2]
#prentugaur1[0,0] = approx[4,4]
#prentugaur1[0,1] = approx[4,5]

print('Part of approximate hitting times\n',approx[0:2,0:2])

print('Part of approximate hitting times\n',not_approx[0:2,0:2])

In [ ]:
# Tafla 10.08.17
mod = Model(dangles='none')
w = State('GUCGCGUCGCGUCGCUAUGCGAC',kind='slow',model=mod)
states,pi,R = w.rate_matrix()
R /= 1e6
big_Pi = np.asarray([pi] * len(states))


Z_exact = -(np.linalg.inv(R + big_Pi) - big_Pi)
Gamma_exact = np.diag(pi) @ Z_exact
S_exact = (Z_exact @ np.diag(1/pi))[:len(states),:len(states)]
Hitting_times_exact = -S_exact + np.asarray([np.diagonal(S_exact)]*len(S_exact))

# 1-PI
#Pi = np.diag(pi)
#unit_matrix = np.diag(np.ones(len(pi)))
#test = unit_matrix - Pi

#Z = np.asarray([Gamma[i, :] / 2 / pi[i] for i in range(len(pi))])
#print('S\n', S_exact, '\n')
#print('Hitting times\n', -S_exact + np.asarray([np.diagonal(S_exact)]*len(S_exact)))
#print('Gamma\n', Gamma_exact, '\n')
#print(np.linalg.eigvals(Gamma_exact))

print('-------------------------------------------')
print('Cut-down hitting times:')
cut_down = np.zeros([2,2])
for k in range(len(states)):
    if states[k] == State('GTCGCGTCGCGTCGCTATGCGAC', '(((((...((...))...)))))', kind='slow', model=mod):
        nr1 = k
    if states[k] == State('GTCGCGTCGCGTCGCTATGCGAC', '(((((.(.((...)).).)))))', kind='slow',model=mod):
        nr2 = k
cut_down[0,0] = Hitting_times_exact[nr1,nr1]
cut_down[0,1] = Hitting_times_exact[nr2,nr1]
cut_down[1,0] = Hitting_times_exact[nr1,nr2]
cut_down[1,1] = Hitting_times_exact[nr2,nr2]

In [ ]:
print(nr1)
print(nr2)
print(np.shape(Hitting_times_exact))
print(Hitting_times_exact[nr1,nr1])
print(Hitting_times_exact[nr1,nr2])
print(Hitting_times_exact[nr2,nr1])
print(Hitting_times_exact[nr2,nr2])
cut_down[0,1] = Hitting_times_exact[nr2,nr1]
cut_down[1,0] = Hitting_times_exact[nr1,nr2]
print(cut_down)

In [ ]:
(Gamma[1] - Gamma[1] @ big_Pi).sum()

In [ ]:
fix = (np.eye(len(states)) - big_Pi).T @ Gamma @ (np.eye(len(states)) - big_Pi)
print(fix.sum(0))
print(fix.sum(1))

In [ ]:
some_strucs_temp = []
for j in range(len(some_strucs)):
    if all(some_strucs[j]!= some_strucs_temp[k] for k in range(len(some_strucs_temp)) ):
        some_strucs_temp.append(some_strucs[j])

In [ ]:
class Stop_Condition:
    def __init__(self, stop_state):
        self.time = 0.0
        self.stop_state = stop_state
        
    def __call__(self, state, timestep):
        #print(state, timestep)
        self.time += timestep
        return state.dp == self.stop_state.dp
seq = 'AAATTT'
w = '......'    
w_before = w
vigur =[]
Define_State = State(seq, w)
vigur.append(Define_State)
vigur.append(OneStep(Define_State))
Define_State
_State = State(seq,w_after)

stop = Stop_Condition(w_after)
Time = stop.time

In [ ]:
vigur = []
def FriidKallar(w):
    nytt = w
    class Stop_Condition:
        def __init__(self, stop_state):
            self.time = 0.0
            self.stop_state = stop_state
        
        def __call__(self, state, timestep):
            #print(state, timestep)
            self.time += timestep
            return state == self.stop_state


    stop_state = State('AAATTT', '.(...)')
    w = State('AAATTT')
    stop = Stop_Condition(stop_state)
    w.run(stop)
    vigur.append(stop.time)

In [ ]:
class Stop_Condition:
    def __init__(self, stop_state):
        self.time = 0.0
        self.stop_state = stop_state
        
    def __call__(self, state, timestep):
            #print(state, timestep)
        self.time += timestep
        return state == self.stop_state
    mod = Model(dangles='none')

for i in range(100000):
    w = State('GTCGCGTCGCGTCGCTATGCGAC', '(((((...((...))...)))))', kind='slow', model=mod)
    stop_state = State('GTCGCGTCGCGTCGCTATGCGAC', '(((((.(.((...)).).)))))', kind='slow',model=mod)
    stop = Stop_Condition(stop_state)
    vigur.append(stop.time)
np.mean(vigur)*1e6

In [ ]:
######################################################################################################################

                                          # Current version
                                           # @Mark Fornace

#####################################################################################################################

In [ ]:
import numpy as np
from scipy.linalg import eigh
from nupack4 import *

# Define the model used for all the states (temperature and gt-pairs, dangles always have to be none)
mod = Model(dangles='none',gt = True)

# System defined
w = State('GCGUCGCGUCGCUAUGC', '.....((((....))))',kind='slow', model=mod)
#w = State('AUCUGGUCCCAAGUUUCCCCGAUCGUAGGAUCGGCCCAAAUUAACAGGGUCACUCAUCGGAGUGAGGAAACAUCCAGCCGGAGGUCCGGCUGGGGGACCAGU',kind='slow',model=mod)
#w = State('GUCGCGUCGCGUCGCUAUGCGAC',kind='slow',model=mod)
#w = State('AGTCTAGGATTCGGCGTGGGTTAACACGCCGAATCCTAGACTACTTTG+AGTCTAGGATTCGGCGTGGGTTAA+TTAACCCACGCCGAATCCTAGACTCAAAGTAGTCTAGGATTCGGCGTG',kind='slow',model=mod)
#w = State('AAATTT',kind='slow',model=mod)

# Gets rid of duplicates in basis states
def unique_states(states):
    tmp = []
    for j in range(len(states)):
        if all(states[j]!= tmp[k] for k in range(len(tmp)) ):
            tmp.append(states[j])
    return tmp
       
# Sampling using the thermo code
def sample(eng, w, n):
    seqs = w.sequence.split('+')
    nicks = [0]
    for s in seqs:
        nicks.append(nicks[-1] + len(s))
    return [State(w.sequence, s.dp(nicks[1:]), model=mod, kind='slow') for s in eng.sample(w.sequence, n)]

eng = Engine(model=mod)

# Get basis_states to define basis.Many possible methods (both for selection and clustering)

#basis_states = sample(eng, w, 50)
#basis_states = w.rate_matrix()[0][:2]
#basis_states = unique_states(basis_states)

basis_states = []
basis_dp = ['.....((((....))))','((((.((...)).))))','(((......))).....']
#basis_dp = ['((...))..........','(((......))).....','((.(.((...)).).))']
#basis_dp = ['..(((...)))((((...)))).', '...((...))(((((...)))))', '..........(((((...)))))', '(((((((.((...)).)))))))','.....((((((......))))))']
for i in range(len(basis_dp)):
    basis_states.append(State('GCGUCGCGUCGCUAUGC', basis_dp[i],kind='slow',model=mod))
    
    
launch_states = sample(eng, w, 10000)
    
basis_strucs = [y.pairs for y in basis_states]

n = len(basis_strucs)
print('Without duplicates:', n)
print(basis_states)


# Set the scale (lambda) and time.
scale = 1e7
t = 1e-3
A = slice(None)
B = slice(n, None)

Gamma = np.zeros([2*n, 2 *n])
time = 0.
Overlap = np.zeros([2*n, 2 *n])
Grad = np.zeros([2*n, 2 *n])
times = np.zeros([2*n])


Get the lunch states by sampling code
def get_launch_states(engine, n):
    pass

for z in launch_states:
    #enum = enumerated_integrator(enumerated_observer(pairs, np.eye(n)), scale, 0)
    ham = hamming_integrator(hamming_observer(basis_strucs, 0), scale, 0)
    #cov = covariance_integrator(lambda w, t: np.ones([4]), scale, 0)
    z.run(time=t, observers=[ham])
    Gamma += ham.gamma
    time += ham.time
    times += ham.times
    #print(ham.times)
    Overlap += ham.overlap
    Grad += ham.gradient
    
print(t, '\t', 1/scale)

#print(enum.gamma)
Overlap = Overlap[A, A] / time # Matrix
Overlap = 0.5 * (Overlap + Overlap.T)
print('Overlap\n', Overlap)

norms = np.sqrt(np.diagonal(Overlap))
print('Norms', norms)
Ham_Gamma = 1e6 * Gamma[A, A] / time  / np.outer(norms, norms) # Matrix
Ham_Gamma = 0.5 * (Ham_Gamma + Ham_Gamma.T)
print('Gamma eigs\n', np.linalg.eigvals(Ham_Gamma))

Ham_Grad = 1e12 * Grad[A, A] / time / np.outer(norms, norms)
Ham_Grad = 0.5 * (Ham_Grad + Ham_Grad.T)
print('Gradient\n', Ham_Grad)

# Different way, tried on 08/23/2017, this way no gamma2 is needed, only plug in the fix value in corresponding places.
Ham_Gamma_Inv = np.linalg.ing(Ham_Gamma)
Ham_Gamma_fix = np.linalg.inv(Ham_Gamma_Inv - (scale/1e6) * Ham_Gamma_Inv @ Ham_Grad @ Ham_Gamma_Inv)

Norm_Overlap = Overlap / np.outer(norms, norms)

print('Normalized overlap\n', Norm_Overlap)

means = times[A] / time / norms # vector
iGamma = np.linalg.inv(Ham_Gamma) # matrix
new_pi = means * (iGamma @ means) # vector
eta = new_pi.sum()   # scalar
new_pi /= eta

#print('stuff', means @ (iGamma @ means), means @ iGamma @ means, (means * (iGamma @ means)).sum())
print('stationary state', new_pi.shape, new_pi.sum(), '\n', new_pi)

M = np.outer(means, means) / eta
Projected_Gamma = (np.eye(len(Ham_Gamma)) - iGamma @ M).T @ Ham_Gamma @ (np.eye(len(Ham_Gamma)) - iGamma @ M)

Ham_S = np.diag(1/means[B]) @ Projected_Gamma[B, B] @ np.diag(1/means[B])

gamma_pi = iGamma @ means / eta

Gamma_Pi = np.outer(gamma_pi, gamma_pi) / (gamma_pi @ gamma_pi)

T = np.linalg.inv(Projected_Gamma + Gamma_Pi) - Gamma_Pi

#printout only for checking
print('1st order T\n', T)
print('T correction\n', (scale / 1e6) * T @ Ham_Grad @ T)

Gamma2 = np.linalg.inv(Gamma_Pi + T + (scale/1e6) * T @ Ham_Grad @ T) - Gamma_Pi
# printout only for checking
print('Gamma eigs\n', np.linalg.eigvals(Projected_Gamma))
print('T eigs\n', np.linalg.eigvals(T))
print('Gamma2 eigs\n', np.linalg.eigvals(Gamma2))

print('------------------------------------------')
print('should be 0\n', Projected_Gamma @ gamma_pi)
print('-')
print(T@gamma_pi)
print('-')
print(Gamma2@gamma_pi)
print('------------------------------------------')

print('Approximate Gamma\n', Projected_Gamma)
print('Fixed Gamma\n', Gamma2)

iGamma2 = np.linalg.inv(Gamma2[B, B])
new_pi2 = means[B] * (iGamma2 @ means[B]) # vector
eta2 = new_pi2.sum()
new_pi2 /= eta2
M2 = np.outer(means[B], means[B]) / eta2
Projected_Gamma2 = (np.eye(len(iGamma2)) - iGamma2 @ M2).T @ Gamma2[B, B] @ (np.eye(len(iGamma2)) - iGamma2 @ M2)
Ham_S2 = np.diag(1/means[B]) @ Projected_Gamma2 @ np.diag(1/means[B])

# printout only for checking
print('Approximate S\n', Ham_S)
print('Fixed S\n', Ham_S2)
print('Approximate hitting time\n', -Ham_S + np.asarray([np.diagonal(Ham_S)]*len(Ham_S)))
print('Fixed hitting time\n', -Ham_S2 + np.asarray([np.diagonal(Ham_S2)]*len(Ham_S2)))


approx = -Ham_S + np.asarray([np.diagonal(Ham_S)]*len(Ham_S))
not_approx = -Ham_S2 + np.asarray([np.diagonal(Ham_S2)]*len(Ham_S2))

# Let's find the rate matrix (R reduced)
print('Steady S', Ham_S2 @ new_pi2)
Z_2 = Ham_S2 @ np.diag(new_pi2)

print('S eigs', np.linalg.eigvals(Ham_S2))

big_pi2 = np.zeros([len(new_pi2),len(new_pi2)]) + new_pi2



R_2 = np.linalg.inv(Z_2+big_pi2)-big_pi2
print('R eigs', np.linalg.eigvals(R_2))
print('Z eigs', np.linalg.eigvals(Z_2))



print('Fixed rate matrix\n', R_2)
print('New pi', new_pi2)
#print('------------------------------')
#prentugaur1 = np.zeros[2,2]
#prentugaur1[0,0] = approx[4,4]
#prentugaur1[0,1] = approx[4,5]

#print('Part of approximate hitting times\n',approx[0:2,0:2])

#print('Part of approximate hitting times\n',not_approx[0:2,0:2])

In [ ]:
# Code for multistand cases 


eng = Engine(model=mod)
mod = Model(dangles='none',gt = True)
mod.molarity = 1.e-6

def sample(n, eng, seq, model):
    return [State(seq, s.dp([len(seq)]), model=model, kind='slow') for s in eng.sample(seq, n)]


def calculate_pf(mod, perm, strands):
    ret = 1.
    for i in perm:
        seq = '+'.join(strands[j] for j in i)
        ret *= eng(seq)
    scale = (mod.unimolecular_scaling / mod.bimolecular_scaling / mod.molarity) ** (len(perm) - len(strands))
    return ret * scale

seqs = 'GUCGCGUCGCGUCGCUAUGCGAC'.split('+')

partitions = unique_partitions(range(len(seqs)))


pfs = np.asarray([calculate_pf(mod, p, seqs) for p in partitions])
pfs /= pfs.sum()

for p, pf in zip(partitions, pfs):
    print(pf, ' '.join('+'.join(str(i) for i in j) for j in p))
    
n_launch = 5
bias_pfs = pfs ** 1.0
bias_pfs /= bias_pfs.sum()
launch_indices = np.random.choice(len(partitions), p=bias_pfs, size=n_launch)
launch_perms = [partitions[i] for i in launch_indices]
weights = [pfs[i] / bias_pfs[i] for i in launch_indices]

def perm_to_state(perm):
    # where each strand starts in standard representation
    nicks = [1]
    for s in seqs:
        nicks.append(nicks[-1] + len(s) + 2) # where each strand starts, but 2 null bases between each strand
    
    # where each strand starts in this representation
        
    ret = np.arange(sum(len(s) for s in seqs) + 2 * len(seqs)) # standard pair vec
    
    for cx in perm:
        pairs = np.asarray(eng.sample('+'.join(seqs[i] for i in cx))[0]) # no null bases included
        
        transfer = [] # where bases are supposed to go
        for s in cx:
            for i in range(len(seqs[s])):
                transfer.append(nicks[s] + i)
        transfer = np.asarray(transfer, dtype=int)
        
        start = 0
        for s in cx:
            ret[nicks[s]:nicks[s]+len(seqs[s])] = transfer[pairs[start:start+len(seqs[s])]]
            start += len(seqs[s])
    
    pairs = State('+'.join(seqs)).pairs
    for i, j in enumerate(ret):
        pairs[i] = j
    
    return State('+'.join(seqs), model=mod, kind='slow').with_structure(pairs)
    
        
    
print(perm_to_state(launch_perms[0]))
    

In [ ]:
p = State('AAAAT', '.....')
p2 = State('AAAAT', '(...)')

def hamming_distance(w1, w2):
    return np.sum(w1.pairs.array() != w2.pairs.array())

hamming_distance(p,p2)

In [ ]:
# To compare enumerated case to new algorithm
mod = Model(dangles='none')
w = State('GUCGCGUCGCGUCGCUAUGCGAC',kind='slow',model=mod)
states,pi,R = w.rate_matrix()
R /= 1e6
big_Pi = np.asarray([pi] * len(states))


Z_exact = -(np.linalg.inv(R + big_Pi) - big_Pi)
Gamma_exact = np.diag(pi) @ Z_exact
S_exact = (Z_exact @ np.diag(1/pi))[:len(states),:len(states)]
Hitting_times_exact = -S_exact + np.asarray([np.diagonal(S_exact)]*len(S_exact))


print('Rate matrix:\n',R)

# 1-PI
#Pi = np.diag(pi)
#unit_matrix = np.diag(np.ones(len(pi)))
#test = unit_matrix - Pi

#Z = np.asarray([Gamma[i, :] / 2 / pi[i] for i in range(len(pi))])
#print('S\n', S_exact, '\n')
#print('Hitting times\n', -S_exact + np.asarray([np.diagonal(S_exact)]*len(S_exact)))
#print('Gamma\n', Gamma_exact, '\n')
#print(np.linalg.eigvals(Gamma_exact))

#print('-------------------------------------------')
#print('Cut-down hitting times:')
#cut_down = np.zeros([2,2])
#for k in range(len(states)):
#    if states[k] == State('GUCGCGUCGCGUCGCUAUGCGAC', '(((((...((...))...)))))', kind='slow', model=mod):
#        nr1 = k
#    if states[k] == State('GUCGCGUCGCGUCGCUAUGCGAC', '(((((.(.((...)).).)))))', kind='slow',model=mod):
#        nr2 = k
#cut_down[0,0] = Hitting_times_exact[nr1,nr1]
#cut_down[0,1] = Hitting_times_exact[nr2,nr1]
#cut_down[1,0] = Hitting_times_exact[nr1,nr2]
#cut_down[1,1] = Hitting_times_exact[nr2,nr2]

In [ ]:
print(nr1)
print(nr2)
print(np.shape(Hitting_times_exact))
print(Hitting_times_exact[nr1,nr1])
print(Hitting_times_exact[nr1,nr2])
print(Hitting_times_exact[nr2,nr1])
print(Hitting_times_exact[nr2,nr2])
cut_down[0,1] = Hitting_times_exact[nr2,nr1]
cut_down[1,0] = Hitting_times_exact[nr1,nr2]
print(cut_down)

In [ ]:
mod = Model(dangles='none',gt = True)
basis_states = []
basis_dp = ['..(((...)))((((...)))).', '...((...))(((((...)))))', '..........(((((...)))))', '(((((((.((...)).)))))))','.....((((((......))))))']
for i in range(len(basis_dp)):
    basis_states.append(State('GUCGCGUCGCGUCGCUAUGCGAC', basis_dp[i],kind='slow',model=mod))

In [ ]:
Z_2 = Ham_S2*new_pi2
Z_2 = Ham_S2*new_pi2

big_pi2 = np.zeros([len(new_pi2),len(new_pi2)]) + new_pi2



R_2 = np.linalg.inv(Z_2+big_pi2)-big_pi2

In [ ]:
act